# Training and cross validation of symbolic regression model

This workbook performs symbolic regression (SR) on bulk modulus data. 

The SR algorithm was first implemented by Flores et. al. in the following work DOI: [10.1039/D2DD00027J](https://doi.org/10.1039/D2DD00027J).

In [1]:
# import packages
import os, sys, platform
import numpy as np
import pandas as pd
import workflows as wf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import sklearn.metrics as skmetrics 
from itertools import permutations, combinations
import runSR as sr

print('Python version', sys.version)
print('Running on', platform.system())

Python version 3.8.3 (default, May 19 2020, 13:54:14) 
[Clang 10.0.0 ]
Running on Darwin


In [2]:
# LOADING THE TRAINING DATA
#=========================================
training_file = 'Data_Summary/BulkModulus_Data.csv' #Everything is in SI units now
data_df = pd.read_csv(training_file)
data_df['d'] = data_df['d']/1E9 # Convert Pa to GPa for coefficient magnitude

In [3]:
# GENERATE KFOLD INDICES
#=========================================

splits = 10
kf = KFold(n_splits = splits, shuffle = True, random_state = 1)
result = next(kf.split(data_df), None)
Ntrain = len(data_df.iloc[result[0]])
Ntest =  len(data_df.iloc[result[1]])
print('Test size:')
print(Ntest)
print('Training size:')
print(Ntrain)

Test size:
17
Training size:
151


In [4]:
# GET EXPRESSION COMBINATION LIST
#=========================================

expressions = ["sqrt","^2","^3"]
expression_array = []

for i in range(len(expressions)):
    exp = list(combinations(expressions, i+1))
    expression_array = expression_array + exp

expression_array = [list(exp_tup) for exp_tup in expression_array]

In [5]:
# FEATURE ARRAY LIST
#=========================================

feature_array = [1,2,3]

In [6]:
# RUNNING KFOLD LOOPING
#=========================================

df = pd.DataFrame()

for split in range(splits):
    
    result = list(kf.split(data_df))[split]
    train = data_df.iloc[result[0]].reset_index(drop=True)
    test =  data_df.iloc[result[1]].reset_index(drop=True)
    kfold = split
    
    for expressions in expression_array:
    
        for FEATENG_STEPS in feature_array:

            eqn, eqn_str, coeffs, rmse, r2, mape, y_real, y_hat, trained_workflow = sr.run_SR(train,FEATENG_STEPS,expressions)

            #Check test data
            ytest_real = test['d'] #soundspeed or density
            ytest_hat = trained_workflow.predict(x = test[['T','r','m']])
            rmse_test = np.sqrt(skmetrics.mean_squared_error(ytest_real, ytest_hat))
            r2_test = skmetrics.r2_score(ytest_real, ytest_hat)

            single_dict = {'kfold':[kfold],
                           'exp': [expressions],
                           'feats': [FEATENG_STEPS],
                           'eqn': [eqn],
                           'rmse_test': [rmse_test],
                           'r2_test': [r2_test],
                           'rmse_train': [rmse],
                           'r2_train': [r2],
                           'ytrain_real': [[y_real]],
                           'ytrain_hat': [[y_hat]],
                           'ytest_real': [[ytest_real]],
                           'ytest_hat': [[ytest_hat]]}

            df_new = pd.DataFrame.from_dict(single_dict)
            df = pd.concat([df,df_new],axis=0)

[AutoFeat] Warning: This just calls fit_transform() but does not return the transformed dataframe.
[AutoFeat] It is much more efficient to call fit_transform() instead of fit() and transform()!
[AutoFeat] The 1 step feature engineering process could generate up to 3 features.
[AutoFeat] With 151 data points this new feature matrix would use about 0.00 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 3 transformed features from 3 original features - done.
[feateng] Generated altogether 3 new features in 1 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 0 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 3 features after 5 feature selection runs
[featsel] 3 features after correlation filtering

/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(


[feateng] Generated 3 transformed features from 3 original features - done.
[feateng] Step 2: first combination of features
[feateng] Generated 60 feature combinations from 15 original feature tuples - done.
[feateng] Step 3: transformation of new features
[feateng] Generated 40 transformed features from 60 original features - done.
[feateng] Generated altogether 103 new features in 3 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 5 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 8 features after 5 feature selection runs
[featsel] 5 features after correlation filtering
[featsel] 5 features after noise filtering
[AutoFeat] Computing 2 new features.
[AutoFeat]     2/    2 new features ...done.
[AutoFeat] Final dataframe with 5 

/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(


[feateng] Generated 3 transformed features from 3 original features - done.
[feateng] Generated altogether 3 new features in 1 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 0 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 3 features after 5 feature selection runs
[featsel] 3 features after correlation filtering
[featsel] 3 features after noise filtering
[AutoFeat] Final dataframe with 3 feature columns (0 new).
[AutoFeat] Training final regression model.
[AutoFeat] Trained model: largest coefficients:
5.310456053882697
1.051961 * r
0.185017 * m
-0.013136 * T
[AutoFeat] Final score: 0.9927
[Sparsification] Loop: 0. Current number of features: 3
TRAINING COMPLETE
[AutoFeat] Warning: This just calls fit_transform() but does no

...

/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(


[feateng] Generated 6 transformed features from 3 original features - done.
[feateng] Generated altogether 6 new features in 1 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 2 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 3 features after 5 feature selection runs
[featsel] 3 features after correlation filtering
[featsel] 3 features after noise filtering
[AutoFeat] Final dataframe with 3 feature columns (0 new).
[AutoFeat] Training final regression model.
[AutoFeat] Trained model: largest coefficients:
5.306596409520121
1.050891 * r
0.182824 * m
-0.013114 * T
[AutoFeat] Final score: 0.9930
[Sparsification] Loop: 0. Current number of features: 3
TRAINING COMPLETE
[AutoFeat] Warning: This just calls fit_transform() but does no

/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(


[feateng] Generated 9 transformed features from 3 original features - done.
[feateng] Generated altogether 9 new features in 1 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 2 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 3 features after 5 feature selection runs
[featsel] 3 features after correlation filtering
[featsel] 3 features after noise filtering
[AutoFeat] Final dataframe with 3 feature columns (0 new).
[AutoFeat] Training final regression model.
[AutoFeat] Trained model: largest coefficients:
5.306596409520121
1.050891 * r
0.182824 * m
-0.013114 * T
[AutoFeat] Final score: 0.9930
[Sparsification] Loop: 0. Current number of features: 3
TRAINING COMPLETE
[AutoFeat] Warning: This just calls fit_transform() but does no

/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/howiechu/miniconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised i

In [11]:
# SAVE AND SEE RESULTS
#=========================================
export_filename='SR_Kfold_BulkModulus_Results.csv'                             
df.to_csv(export_filename) 